In [ ]:
# This script return the mAE and mAP performances for specific models.
# PREAMBLE: 
# replace iou_thresh=0.5 by iou_thresh=0.001 in maskrcnn_benchmark/data/datasets/evaluation/voc/voc_eval.py L29 
# for a fair comparison with DISCount (they do not consider the complete radicale but only it's root, hence low IoU)

In [1]:
from tqdm import tqdm
import cv2
import numpy as np
from demo.predictor import COCODemo
from maskrcnn_benchmark.utils.env import setup_environment  # noqa F401 isort:skip

from PIL import Image

import argparse
import os

import torch

from torchvision import transforms as T
from torchvision.transforms import functional as F
from maskrcnn_benchmark.config import cfg
from maskrcnn_benchmark.data import make_data_loader
from maskrcnn_benchmark.data.datasets import FolderDataset
from maskrcnn_benchmark.engine.inference import inference
from maskrcnn_benchmark.modeling.detector import build_detection_model
from maskrcnn_benchmark.utils.checkpoint import DetectronCheckpointer
from maskrcnn_benchmark.utils.collect_env import collect_env_info
from maskrcnn_benchmark.utils.comm import synchronize, get_rank
from maskrcnn_benchmark.utils.logger import setup_logger
from maskrcnn_benchmark.utils.miscellaneous import mkdir
from maskrcnn_benchmark.utils.imports import import_file

# from predictor import Cococompute_prediction, select_top_predictions, Resize
# Check if we can enable mixed-precision via apex.amp
try:
    from apex import amp
except ImportError:
    raise ImportError('Use APEX for mixed precision via apex.amp')


model_name = "R_50_C4_1x"
run = 19
Results = {}

config_file = f"configs/e2e_faster_rcnn_{model_name}.yaml" # args.config_file #"../configs/caffe2/e2e_mask_rcnn_R_50_FPN_1x_caffe2.yaml"

model_name = "R_50_C4_1x_pre"
# update the config options with the config file
cfg.merge_from_file(config_file)
# manual override some options
cfg.merge_from_list(["MODEL.DEVICE", "cpu"])
#     cfg.merge_from_list(args.opts)

# cfg.DATASETS.DATA_DIR = data
cfg.TEST.IMS_PER_BATCH = 1 
cfg.TEST.DETECTIONS_PER_IMG = 200 
cfg.MODEL.ROI_HEADS.DETECTIONS_PER_IMG = 200 
cfg.MODEL.ROI_BOX_HEAD.NUM_CLASSES = 3 
cfg.MODEL.RETINANET.NUM_CLASSES = 3 

cfg.MODEL.ROI_HEADS.NMS = 0.1
cfg.DATASETS.DATA_DIR = "/media/giancos/Football/CloudLabeling_DS/CloudLabeling/Seeds_Striga_Strategy1" #${data}
cfg.DATASETS.TEST=('folder_test',)
# print(cfg)
cfg.MODEL.DEVICE = "cuda:0"
    
    
data_loaders_val = make_data_loader(cfg, is_train=False, is_distributed=False)
print(data_loaders_val)

from maskrcnn_benchmark.engine.inference import inference, compute_on_dataset, _accumulate_predictions_from_multiple_gpus
model = build_detection_model(cfg)
model.to(cfg.MODEL.DEVICE)


from maskrcnn_benchmark.utils.checkpoint import DetectronCheckpointer
cfg.OUTPUT_DIR=f"{cfg.DATASETS.DATA_DIR}/output/{model_name}_{run}"
output_dir = cfg.OUTPUT_DIR
checkpointer = DetectronCheckpointer(cfg, model, save_dir=output_dir)
ckpt=f"{cfg.OUTPUT_DIR}/model_bestval.pth"
print(cfg.MODEL.WEIGHT,ckpt)

# ckpt = cfg.MODEL.WEIGHT # if args.ckpt is None else args.ckpt
# "${data}/output/${model_name}_${run}/model_bestval.pth"
_ = checkpointer.load(ckpt, use_latest=False)
    
iou_types = ("bbox",)
dataset_names = cfg.DATASETS.TEST
output_folder = "butta"

predictions = compute_on_dataset(model, data_loaders_val[0], cfg.MODEL.DEVICE, cfg.TEST.BBOX_AUG.ENABLED, None)
predictions=_accumulate_predictions_from_multiple_gpus(predictions)


from maskrcnn_benchmark.data.datasets.evaluation import evaluate
extra_args = dict(
        box_only=False if cfg.MODEL.RETINANET_ON else cfg.MODEL.RPN_ONLY,
        iou_types=iou_types,
        expected_results=cfg.TEST.EXPECTED_RESULTS,
        expected_results_sigma_tol=cfg.TEST.EXPECTED_RESULTS_SIGMA_TOL,
    )
results = evaluate(dataset=data_loaders_val[0].dataset,
        predictions=predictions,
        output_folder=output_folder,
        **extra_args)
print(results)

['__background__', 'Germinated', 'Non-germinated']
catalog://ImageNetPretrained/MSRA/R-50 /media/giancos/Football/CloudLabeling_DS/CloudLabeling/Seeds_Striga_Strategy1/output/R_50_C4_1x_pre_19/model_bestval.pth


100%|██████████| 32/32 [00:08<00:00,  4.30it/s]


nb_classes 3
{'ap': array([       nan, 0.89591617, 0.89475135]), 'map': 0.8953337573543965, 'ae': array([0.        , 0.11624797, 0.06324614]), 'mae': 0.08974705202543778}


In [2]:
from maskrcnn_benchmark.structures.bounding_box import BoxList

pred_discord = []
for img in data_loaders_val[0].dataset.img_files:
#     print(img)
    discount_pred_file = os.path.basename(img).replace(".jpg",".txt")
    discount_pred_file = os.path.join("/media/giancos/Football/CloudLabeling_DS/Discount_Striga",discount_pred_file)
    boxes = []
    labels = []
    scores = []
    with open(discount_pred_file,"r") as file:
        
        data = file.read()
        lines = data.split("\n")
        for line in lines:
#             print("line",line)
            line = line.split(",")
            if len(line) < 4:
                break
            boxes.append([float(line[0]),float(line[1]),float(line[2]),float(line[3])])
            if int(float(line[5])) == 0:
                labels.append(2)
            elif int(float(line[5])) == 1:
                labels.append(1)
            scores.append(float(line[4]))
#         print()
#     print(boxes)
    boxlist = BoxList(boxes, (2592, 1944), mode="xyxy")
    boxlist.add_field("labels", torch.tensor(labels))
    boxlist.add_field("scores", torch.tensor(scores))
    boxlist = boxlist.resize((1066, 800))
#     print(boxlist)
    pred_discord.append(boxlist)
print(pred_discord[0])

from maskrcnn_benchmark.data.datasets.evaluation import evaluate
extra_args = dict(
        box_only=False if cfg.MODEL.RETINANET_ON else cfg.MODEL.RPN_ONLY,
        iou_types=iou_types,
        expected_results=cfg.TEST.EXPECTED_RESULTS,
        expected_results_sigma_tol=cfg.TEST.EXPECTED_RESULTS_SIGMA_TOL,
    )
results = evaluate(dataset=data_loaders_val[0].dataset,
        predictions=pred_discord,
        output_folder="butta_discord",
        **extra_args)
print(results)

BoxList(num_boxes=82, image_width=1066, image_height=800, mode=xyxy)
nb_classes 3
{'ap': array([       nan, 0.26668617, 0.63879713]), 'map': 0.45274164651181736, 'ae': array([0.        , 0.73130263, 0.34772305]), 'mae': 0.5395128411911534}


In [ ]:
# OROBANCHE

In [3]:
from tqdm import tqdm
import cv2
import numpy as np
from demo.predictor import COCODemo
from maskrcnn_benchmark.utils.env import setup_environment  # noqa F401 isort:skip

from PIL import Image

import argparse
import os

import torch

from torchvision import transforms as T
from torchvision.transforms import functional as F
from maskrcnn_benchmark.config import cfg
from maskrcnn_benchmark.data import make_data_loader
from maskrcnn_benchmark.data.datasets import FolderDataset
from maskrcnn_benchmark.engine.inference import inference
from maskrcnn_benchmark.modeling.detector import build_detection_model
from maskrcnn_benchmark.utils.checkpoint import DetectronCheckpointer
from maskrcnn_benchmark.utils.collect_env import collect_env_info
from maskrcnn_benchmark.utils.comm import synchronize, get_rank
from maskrcnn_benchmark.utils.logger import setup_logger
from maskrcnn_benchmark.utils.miscellaneous import mkdir
from maskrcnn_benchmark.utils.imports import import_file

# from predictor import Cococompute_prediction, select_top_predictions, Resize
# Check if we can enable mixed-precision via apex.amp
try:
    from apex import amp
except ImportError:
    raise ImportError('Use APEX for mixed precision via apex.amp')


model_name = "R_50_C4_1x"
run = 19
Results = {}

config_file = f"configs/e2e_faster_rcnn_{model_name}.yaml" # args.config_file #"../configs/caffe2/e2e_mask_rcnn_R_50_FPN_1x_caffe2.yaml"

model_name = "R_50_C4_1x_pre"
# update the config options with the config file
cfg.merge_from_file(config_file)
# manual override some options
cfg.merge_from_list(["MODEL.DEVICE", "cpu"])
#     cfg.merge_from_list(args.opts)

# cfg.DATASETS.DATA_DIR = data
cfg.TEST.IMS_PER_BATCH = 1 
cfg.TEST.DETECTIONS_PER_IMG = 200 
cfg.MODEL.ROI_HEADS.DETECTIONS_PER_IMG = 200 
cfg.MODEL.ROI_BOX_HEAD.NUM_CLASSES = 3 
cfg.MODEL.RETINANET.NUM_CLASSES = 3 

cfg.MODEL.ROI_HEADS.NMS = 0.1
cfg.DATASETS.DATA_DIR = "/media/giancos/Football/CloudLabeling/Seeds_Orobanche_Strategy1" #${data}
cfg.DATASETS.TEST=('folder_test',)
# print(cfg)
cfg.MODEL.DEVICE = "cuda:0"

    
data_loaders_val = make_data_loader(cfg, is_train=False, is_distributed=False)
print(data_loaders_val)

from maskrcnn_benchmark.engine.inference import inference, compute_on_dataset, _accumulate_predictions_from_multiple_gpus
model = build_detection_model(cfg)
model.to(cfg.MODEL.DEVICE)


from maskrcnn_benchmark.utils.checkpoint import DetectronCheckpointer
cfg.OUTPUT_DIR = "/media/giancos/Football/CloudLabeling_DS/CloudLabeling/Seeds_Striga_Strategy1" #${data}
cfg.OUTPUT_DIR=f"{cfg.OUTPUT_DIR}/output/{model_name}_{run}"
output_dir = cfg.OUTPUT_DIR
checkpointer = DetectronCheckpointer(cfg, model, save_dir=output_dir)
ckpt=f"{cfg.OUTPUT_DIR}/model_bestval.pth"
print(cfg.MODEL.WEIGHT,ckpt)

# ckpt = cfg.MODEL.WEIGHT # if args.ckpt is None else args.ckpt
# "${data}/output/${model_name}_${run}/model_bestval.pth"
_ = checkpointer.load(ckpt, use_latest=False)
    
iou_types = ("bbox",)
dataset_names = cfg.DATASETS.TEST
output_folder = "butta"

predictions = compute_on_dataset(model, data_loaders_val[0], cfg.MODEL.DEVICE, cfg.TEST.BBOX_AUG.ENABLED, None)
predictions=_accumulate_predictions_from_multiple_gpus(predictions)


from maskrcnn_benchmark.data.datasets.evaluation import evaluate
extra_args = dict(
        box_only=False if cfg.MODEL.RETINANET_ON else cfg.MODEL.RPN_ONLY,
        iou_types=iou_types,
        expected_results=cfg.TEST.EXPECTED_RESULTS,
        expected_results_sigma_tol=cfg.TEST.EXPECTED_RESULTS_SIGMA_TOL,
    )
results = evaluate(dataset=data_loaders_val[0].dataset,
        predictions=predictions,
        output_folder=output_folder,
        **extra_args)
print(results)

['__background__', 'Germinated', 'Non-germinated']


  0%|          | 0/61 [00:00<?, ?it/s]

catalog://ImageNetPretrained/MSRA/R-50 /media/giancos/Football/CloudLabeling_DS/CloudLabeling/Seeds_Striga_Strategy1/output/R_50_C4_1x_pre_19/model_bestval.pth


100%|██████████| 61/61 [00:14<00:00,  4.21it/s]


nb_classes 3
{'ap': array([       nan, 0.85931981, 0.89815732]), 'map': 0.8787385629471126, 'ae': array([0.        , 0.91635579, 0.15221725]), 'mae': 0.534286520996546}


In [4]:
from maskrcnn_benchmark.structures.bounding_box import BoxList

pred_discord = []
for img in data_loaders_val[0].dataset.img_files:
#     print(img)
    discount_pred_file = os.path.basename(img).replace(".jpg",".txt")
    discount_pred_file = os.path.join("/media/giancos/Football/CloudLabeling_DS/Discount_Orobanche",discount_pred_file)
    boxes = []
    labels = []
    scores = []
    with open(discount_pred_file,"r") as file:
        
        data = file.read()
        lines = data.split("\n")
        for line in lines:
#             print("line",line)
            line = line.split(",")
            if len(line) < 4:
                break
            boxes.append([float(line[0]),float(line[1]),float(line[2]),float(line[3])])
            if int(float(line[5])) == 0:
                labels.append(2)
            elif int(float(line[5])) == 1:
                labels.append(1)
            scores.append(float(line[4]))
#         print()
#     print(boxes)
    boxlist = BoxList(boxes, (2592, 1944), mode="xyxy")
    boxlist.add_field("labels", torch.tensor(labels))
    boxlist.add_field("scores", torch.tensor(scores))
    boxlist = boxlist.resize((1066, 800))
#     print(boxlist)
    pred_discord.append(boxlist)
print(pred_discord[0])

from maskrcnn_benchmark.data.datasets.evaluation import evaluate
extra_args = dict(
        box_only=False if cfg.MODEL.RETINANET_ON else cfg.MODEL.RPN_ONLY,
        iou_types=iou_types,
        expected_results=cfg.TEST.EXPECTED_RESULTS,
        expected_results_sigma_tol=cfg.TEST.EXPECTED_RESULTS_SIGMA_TOL,
    )
results = evaluate(dataset=data_loaders_val[0].dataset,
        predictions=pred_discord,
        output_folder=output_folder,
        **extra_args)
print(results)

BoxList(num_boxes=52, image_width=1066, image_height=800, mode=xyxy)
nb_classes 3
{'ap': array([       nan, 0.38550058, 0.64093949]), 'map': 0.5132200322197774, 'ae': array([0.        , 0.43127242, 0.24803767]), 'mae': 0.3396550413802194}


In [5]:
# OROBANCHE (finetuned)

In [9]:
from tqdm import tqdm
import cv2
import numpy as np
from demo.predictor import COCODemo
from maskrcnn_benchmark.utils.env import setup_environment  # noqa F401 isort:skip

from PIL import Image

import argparse
import os

import torch

from torchvision import transforms as T
from torchvision.transforms import functional as F
from maskrcnn_benchmark.config import cfg
from maskrcnn_benchmark.data import make_data_loader
from maskrcnn_benchmark.data.datasets import FolderDataset
from maskrcnn_benchmark.engine.inference import inference
from maskrcnn_benchmark.modeling.detector import build_detection_model
from maskrcnn_benchmark.utils.checkpoint import DetectronCheckpointer
from maskrcnn_benchmark.utils.collect_env import collect_env_info
from maskrcnn_benchmark.utils.comm import synchronize, get_rank
from maskrcnn_benchmark.utils.logger import setup_logger
from maskrcnn_benchmark.utils.miscellaneous import mkdir
from maskrcnn_benchmark.utils.imports import import_file

# from predictor import Cococompute_prediction, select_top_predictions, Resize
# Check if we can enable mixed-precision via apex.amp
try:
    from apex import amp
except ImportError:
    raise ImportError('Use APEX for mixed precision via apex.amp')


model_name = "R_50_C4_1x"
run = 7
Results = {}

config_file = f"configs/e2e_faster_rcnn_{model_name}.yaml" # args.config_file #"../configs/caffe2/e2e_mask_rcnn_R_50_FPN_1x_caffe2.yaml"

model_name = "R_50_C4_1x_pre"
# update the config options with the config file
cfg.merge_from_file(config_file)
# manual override some options
cfg.merge_from_list(["MODEL.DEVICE", "cpu"])
#     cfg.merge_from_list(args.opts)

# cfg.DATASETS.DATA_DIR = data
cfg.TEST.IMS_PER_BATCH = 1 
cfg.TEST.DETECTIONS_PER_IMG = 200 
cfg.MODEL.ROI_HEADS.DETECTIONS_PER_IMG = 200 
cfg.MODEL.ROI_BOX_HEAD.NUM_CLASSES = 3 
cfg.MODEL.RETINANET.NUM_CLASSES = 3 

cfg.MODEL.ROI_HEADS.NMS = 0.1
cfg.DATASETS.DATA_DIR = "/media/giancos/Football/CloudLabeling/Seeds_Orobanche_Strategy1_finetuning_201030" #${data}
cfg.DATASETS.TEST=('folder_test',)
# print(cfg)
cfg.MODEL.DEVICE = "cuda:0"

    
data_loaders_val = make_data_loader(cfg, is_train=False, is_distributed=False)
print(data_loaders_val)

from maskrcnn_benchmark.engine.inference import inference, compute_on_dataset, _accumulate_predictions_from_multiple_gpus
model = build_detection_model(cfg)
model.to(cfg.MODEL.DEVICE)


from maskrcnn_benchmark.utils.checkpoint import DetectronCheckpointer
# cfg.OUTPUT_DIR = "/media/giancos/Football/CloudLabeling_DS/CloudLabeling/Seeds_Orobanche_Strategy1_finetuning_201030" #${data}
cfg.OUTPUT_DIR=f"{cfg.DATASETS.DATA_DIR}/output/{model_name}_{run}"
output_dir = cfg.OUTPUT_DIR
checkpointer = DetectronCheckpointer(cfg, model, save_dir=output_dir)
ckpt=f"{cfg.OUTPUT_DIR}/model_bestval.pth"
print(cfg.MODEL.WEIGHT,ckpt)

# ckpt = cfg.MODEL.WEIGHT # if args.ckpt is None else args.ckpt
# "${data}/output/${model_name}_${run}/model_bestval.pth"
_ = checkpointer.load(ckpt, use_latest=False)
    
iou_types = ("bbox",)
dataset_names = cfg.DATASETS.TEST
output_folder = "butta"

predictions = compute_on_dataset(model, data_loaders_val[0], cfg.MODEL.DEVICE, cfg.TEST.BBOX_AUG.ENABLED, None)
predictions=_accumulate_predictions_from_multiple_gpus(predictions)

from maskrcnn_benchmark.data.datasets.evaluation import evaluate
extra_args = dict(
        box_only=False if cfg.MODEL.RETINANET_ON else cfg.MODEL.RPN_ONLY,
        iou_types=iou_types,
        expected_results=cfg.TEST.EXPECTED_RESULTS,
        expected_results_sigma_tol=cfg.TEST.EXPECTED_RESULTS_SIGMA_TOL,
    )
results = evaluate(dataset=data_loaders_val[0].dataset,
        predictions=predictions,
        output_folder=output_folder,
        **extra_args)
print(results)

['__background__', 'Germinated', 'Non-germinated']


  0%|          | 0/30 [00:00<?, ?it/s]

catalog://ImageNetPretrained/MSRA/R-50 /media/giancos/Football/CloudLabeling/Seeds_Orobanche_Strategy1_finetuning_201030/output/R_50_C4_1x_pre_7/model_bestval.pth


100%|██████████| 30/30 [00:07<00:00,  4.25it/s]


nb_classes 3
{'ap': array([       nan, 0.89712683, 0.90531029]), 'map': 0.9012185573406113, 'ae': array([0.        , 0.19118587, 0.06752096]), 'mae': 0.12935341707860998}


In [11]:
from maskrcnn_benchmark.structures.bounding_box import BoxList

pred_discord = []
for img in data_loaders_val[0].dataset.img_files:
#     print(img)
    discount_pred_file = os.path.basename(img).replace(".jpg",".txt")
    discount_pred_file = os.path.join("/media/giancos/Football/CloudLabeling_DS/Discount_Orobanche",discount_pred_file)
    boxes = []
    labels = []
    scores = []
    with open(discount_pred_file,"r") as file:
        
        data = file.read()
        lines = data.split("\n")
        for line in lines:
#             print("line",line)
            line = line.split(",")
            if len(line) < 4:
                break
            boxes.append([float(line[0]),float(line[1]),float(line[2]),float(line[3])])
            if int(float(line[5])) == 0:
                labels.append(2)
            elif int(float(line[5])) == 1:
                labels.append(1)
            scores.append(float(line[4]))
#         print()
#     print(boxes)
    boxlist = BoxList(boxes, (2592, 1944), mode="xyxy")
    boxlist.add_field("labels", torch.tensor(labels))
    boxlist.add_field("scores", torch.tensor(scores))
    boxlist = boxlist.resize((1066, 800))
#     print(boxlist)
    pred_discord.append(boxlist)
print(pred_discord[0])


from maskrcnn_benchmark.data.datasets.evaluation import evaluate
extra_args = dict(
        box_only=False if cfg.MODEL.RETINANET_ON else cfg.MODEL.RPN_ONLY,
        iou_types=iou_types,
        expected_results=cfg.TEST.EXPECTED_RESULTS,
        expected_results_sigma_tol=cfg.TEST.EXPECTED_RESULTS_SIGMA_TOL,
    )
results = evaluate(dataset=data_loaders_val[0].dataset,
        predictions=pred_discord,
        output_folder=output_folder,
        **extra_args)
print(results)

BoxList(num_boxes=52, image_width=1066, image_height=800, mode=xyxy)
nb_classes 3
{'ap': array([       nan, 0.45454545, 0.65507655]), 'map': 0.5548110005760647, 'ae': array([0.        , 0.3617455 , 0.27975395]), 'mae': 0.32074972211892583}
